In [1]:
#!kill -9 -1

In [2]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:

#%%time


!pip uninstall -q torch -y
# 
!python3 -m pip install -q torch==1.6.0 -f https://download.pytorch.org/whl/torch_stable.html
!python3 -m pip install -q -U tokenizers==0.7.0 > /dev/null
!python3 -m pip install -q -U transformers==3.0.2 > /dev/null
!python3 -m pip install -q -U simpletransformers==0.46.0 > /dev/null


     |████████████████████████████████| 552.8MB 32kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.6.0+cu92 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.6.0+cu92 which is incompatible.


In [ ]:
#!pip install torch==1.6.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

In [ ]:
cast_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LTP/plots_train.csv", delimiter = ',')
cast_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LTP/plots_test.csv", delimiter = ',')
cast_valid = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LTP/plots_valid.csv", delimiter = ',')


In [ ]:
cast_df.columns = ['cast','Plot']
cast_test.columns = ['cast','Plot']
cast_valid.columns = ['cast','Plot']

In [ ]:
print(cast_df.head())

In [ ]:
cast_counts = cast_df.groupby('cast').count().iloc[:,0]

In [ ]:
print(cast_counts)

In [ ]:
#!python3 -m pip install simpletransformers

In [ ]:
from simpletransformers.classification import ClassificationModel

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "use_cuda" : True,
    "max_seq_length": 512,
    "train_batch_size": 16,
    "num_train_epochs": 4,
}

# Create a ClassificationModel
# model = ClassificationModel("roberta", "roberta-base", num_labels=len(cast_counts), args=model_args)
model = ClassificationModel("longformer", "allenai/longformer-base-4096", num_labels=len(cast_counts), args=model_args)
# model = ClassificationModel("xlnet", "xlnet-base-cased", num_labels=len(cast_counts), args=model_args)


In [ ]:
train_df = cast_df
eval_df = cast_test

# Train the model
model.train_model(train_df[["Plot", "cast"]])

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df[["Plot", "cast"]])
print(result)



In [ ]:

predicted_genres_encoded = list(map(lambda x: np.argmax(x), model_outputs))
print(predicted_genres_encoded)

In [ ]:

label_encoder = LabelEncoder()

predicted_genres = list(map(lambda x: np.argmax(x), model_outputs))
eval_gt_labels = eval_df["cast"].tolist()
class_labels = list(range(len(cast_counts)))


plt.figure(figsize=(22,18))
cf_matrix = confusion_matrix(predicted_genres, eval_gt_labels, class_labels)
ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, cmap="YlGnBu")
ax.set_xlabel('Predicted cast', fontsize=20)
ax.set_ylabel('True cast', fontsize=20)
ax.set_title('Confusion Matrix', fontsize=20)
ax.set_xticklabels(class_labels, rotation=90, fontsize=18)
ax.set_yticklabels(class_labels, rotation=0, fontsize=18)

plt.show()

In [ ]:
random_idx = random.randint(0, len(eval_df)-1)
text = eval_df.iloc[random_idx]['Plot']
true_genre = eval_df.iloc[random_idx]['cast']

# Predict with trained multiclass classification model
predicted_genres_example, raw_outputs = model.predict([text])
predicted_genres_example = np.array(predicted_genres_example)

print(f'\nTrue Genre:'.ljust(16,' '), f'{true_genre}\n')
print(f'Predicted Genre: {predicted_genres_example[0]}\n')
print(f'Plot: {text}\n')
print("-------------------------------------------")

In [ ]:
print(len(predicted_genres))
print(len(eval_gt_labels))

In [ ]:
num = 0
den = 0
for i in range(len(eval_gt_labels)):
  if predicted_genres[i] == eval_gt_labels[i]:
    num += 1
  den += 1

print(num/den)